In [1]:
import pandas as pd
import numpy as np
import os
import random
import torch
import torch.nn as nn
import csv
from itertools import islice

data = []

with open('/home/yisheng/data/traces_extractor/traces.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in islice(reader, 10000):
        data.append({
            "pc": row["pc"],
            "delta_in": row["delta_in"],
            "delta_out": row["delta_out"]
        })

print(data)
split_index = int(len(data) * 0.8)
train_data = data[:split_index]
validation_data = data[split_index:]

[{'pc': '140203332779352', 'delta_in': '-57', 'delta_out': '57'}, {'pc': '140727568839168', 'delta_in': '57', 'delta_out': '-57'}, {'pc': '140203333016768', 'delta_in': '-57', 'delta_out': '14'}, {'pc': '140727568842093', 'delta_in': '14', 'delta_out': '-77537'}, {'pc': '140203333008320', 'delta_in': '-77537', 'delta_out': '77547'}, {'pc': '140727568842093', 'delta_in': '77547', 'delta_out': '-77547'}, {'pc': '140203333008544', 'delta_in': '-77547', 'delta_out': '77547'}, {'pc': '140727568842093', 'delta_in': '77547', 'delta_out': '-77547'}, {'pc': '140203333008656', 'delta_in': '-77547', 'delta_out': '77548'}, {'pc': '140727568842093', 'delta_in': '77548', 'delta_out': '-77548'}, {'pc': '140203333009888', 'delta_in': '-77548', 'delta_out': '77548'}, {'pc': '140727568842093', 'delta_in': '77548', 'delta_out': '-77548'}, {'pc': '140203333010448', 'delta_in': '-77548', 'delta_out': '77548'}, {'pc': '140727568842093', 'delta_in': '77548', 'delta_out': '-77548'}, {'pc': '140203333010560', 

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import DataLoader, Dataset
import torch

from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [3]:
class MemoryAccessDataset(Dataset):
    def __init__(self, tokenizer, data):
        self.input_ids = []
        self.attention_masks = []
        for i, trace in enumerate(data):
            sequence = f"PC: {trace['pc']} Page: {trace['delta_in']} Next Page: {trace['delta_out']}"
            encoding = tokenizer(sequence, max_length=512, padding='max_length', truncation=True, return_tensors="pt")
            self.input_ids.append(encoding.input_ids)
            self.attention_masks.append(encoding.attention_mask)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx].squeeze(), self.attention_masks[idx].squeeze()

In [4]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

# Prepare datasets
train_dataset = MemoryAccessDataset(tokenizer, train_data)
validation_dataset = MemoryAccessDataset(tokenizer, validation_data)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=2)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

epochs = 10

model.train()
for epoch in range(epochs):  # Define epochs
    print("epoch ", epoch)
    for batch in train_loader:
        
        inputs, masks = batch
        inputs, masks = inputs.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()



/home/yisheng/.local/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


epoch  0
epoch  1
epoch  2
epoch  3
epoch  4
epoch  5
epoch  6
epoch  7
epoch  8
epoch  9


In [10]:
def predict_next_page_address(model, tokenizer, pc, page_address):
    model.eval()  # Set the model to evaluation mode
    model.to(device)  # Ensure the model is on the correct device
    
    # Create the prompt from the input PC and page address
    prompt = f"PC: {pc} Page: {page_address}"
    
    # Encode the prompt to be suitable for the model
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    
    # Generate the output using the model
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=50, num_return_sequences=1)
    
    # Decode the generated output to text
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the next page address from the predicted text
    predicted_page_address = predicted_text.split()[-1] 

    return predicted_page_address

# Example usage
pc = "140203333008656"  # Example PC value
page_address = "-77547"  # Example current page address
predicted_page_address = predict_next_page_address(model, tokenizer, pc, page_address)
print(f"Predicted Next Page Address: {predicted_page_address}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicted Next Page Address: 77548


In [11]:
# test the accuracy

def test_accuracy(model, tokenizer, data):
    correct_predictions = 0
    total_predictions = 0
    for trace in data:
        pc = trace['pc']
        page_address = trace['delta_in']
        next_page_address = trace['delta_out']
        predicted_page_address = predict_next_page_address(model, tokenizer, pc, page_address)
        if predicted_page_address == next_page_address:
            correct_predictions += 1
        total_predictions += 1
        print(f"Predicted: {predicted_page_address} Actual: {next_page_address}")
    accuracy = correct_predictions / total_predictions
    return accuracy

accuracy = test_accuracy(model, tokenizer, validation_data)
print(f"Accuracy: {accuracy * 100:.2f}%")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicted: 1 Actual: 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicted: -2614 Actual: -2614


...

Predicted: -2614 Actual: -2618


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicted: 102 Actual: 102


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicted: 2516 Actual: 2516
Predicted: -1322 Actual: -1322
Accuracy: 87.90%


In [12]:
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 87.90%


In [13]:
# Save the model
model_path = "/home/yisheng/data/gpt2_trained_model"
model.save_pretrained(model_path)

# Save the tokenizer
tokenizer_path = "/home/yisheng/data/gpt2_trained_tokenizer"
tokenizer.save_pretrained(tokenizer_path)


('/home/yisheng/data/gpt2_trained_tokenizer/tokenizer_config.json',
 '/home/yisheng/data/gpt2_trained_tokenizer/special_tokens_map.json',
 '/home/yisheng/data/gpt2_trained_tokenizer/vocab.json',
 '/home/yisheng/data/gpt2_trained_tokenizer/merges.txt',
 '/home/yisheng/data/gpt2_trained_tokenizer/added_tokens.json')

In [14]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

# Load the model
model = GPT2LMHeadModel.from_pretrained(model_path)

# Ensure the model is in evaluation mode
model.eval()

# Example usage
pc = "140203333008656"  # Example PC value
page_address = "-77547"  # Example current page address
predicted_page_address = predict_next_page_address(model, tokenizer, pc, page_address)
print(f"Predicted Next Page Address: {predicted_page_address}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicted Next Page Address: 77548
